In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[2]") \
    .config("spark.driver.memory", "20g").getOrCreate()

df = spark.read.parquet("./data/song_lyrics.parquet")
df.printSchema()

23/05/21 14:50:19 WARN Utils: Your hostname, thomas-ThinkPad-T490 resolves to a loopback address: 127.0.1.1; using 192.168.31.207 instead (on interface wlp0s20f3)
23/05/21 14:50:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/21 14:50:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- title: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- year: long (nullable = true)
 |-- views: long (nullable = true)
 |-- features: string (nullable = true)
 |-- lyrics: string (nullable = true)
 |-- id: long (nullable = true)
 |-- language_cld3: string (nullable = true)
 |-- language_ft: string (nullable = true)
 |-- language: string (nullable = true)



In [3]:
df.show()

+--------------------+---+---------+----+------+--------------------+--------------------+---+-------------+-----------+--------+
|               title|tag|   artist|year| views|            features|              lyrics| id|language_cld3|language_ft|language|
+--------------------+---+---------+----+------+--------------------+--------------------+---+-------------+-----------+--------+
|           Killa Cam|rap|  Cam'ron|2004|173166|{"Cam\\'ron","Ope...|[Chorus: Opera St...|  1|           en|         en|      en|
|          Can I Live|rap|    JAY-Z|1996|468624|                  {}|[Produced by Irv ...|  3|           en|         en|      en|
|   Forgive Me Father|rap| Fabolous|2003|  4743|                  {}|Maybe cause I'm e...|  4|           en|         en|      en|
|        Down and Out|rap|  Cam'ron|2004|144404|{"Cam\\'ron","Kan...|[Produced by Kany...|  5|           en|         en|      en|
|              Fly In|rap|Lil Wayne|2005| 78271|                  {}|[Intro]\nSo they ...|

In [3]:
df.select("lyrics").show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
df.filter(df.year.between(1960,2023) & (df.language == 'en')).count()

3338537

In [5]:
df.filter(df.title.isNull()).count()

165

In [6]:
df.na.drop(subset=["title"]).count()

5134691

In [2]:
import math
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

@udf(IntegerType())
def decade(year):
    return int(math.trunc(year / 10) * 10)

In [ ]:

# decadeUDf = udf(lambda z: decade(z), IntegerType())

In [3]:

df = df.filter(df.year.between(1960,2023) & (df.language == 'en') &
               (df.artist != "Genius English Translations")) \
    .withColumn("decade", decade(df.year)) \
    .na.drop(subset=["title"])
    

In [4]:
df.filter(df.artist == 'Genius English Translations').count()

0

In [5]:
from pyspark.sql.functions import min

n_sample = df.groupBy("decade").count().select(min("count")).collect()[0]["min(count)"]

In [6]:
from pyspark.sql.functions import col

frac = df.groupBy("decade").count().withColumn("required_n", n_sample / col("count")).drop("count").rdd.collectAsMap()
print(frac)

{1990: 0.18922084741426898, 2020: 0.058177432752782476, 1960: 1.0, 1970: 0.6472620089347395, 1980: 0.4341853232549997, 2000: 0.09333864647672178, 2010: 0.024496668471680267}


In [7]:
sample = df.stat.sampleBy("decade", frac, 42)
sample.groupBy("decade").count().show()

+------+-----+
|decade|count|
+------+-----+
|  1990|42238|
|  2020|42035|
|  1960|42162|
|  1970|42439|
|  1980|41910|
|  2000|42073|
|  2010|42007|
+------+-----+



In [8]:
sample.count()

294864

In [9]:

sample.filter(sample.artist != 'Genius English Translations').count() + sample.filter(sample.artist == 'Genius English Translations').count()

294864

In [12]:
result.coalesce(1).write.format("csv") \
            .option("header",True) \
            .mode('overwrite') \
            .save('data/result.csv')

In [6]:
df_res.to_csv('result.csv')

In [7]:
import pandas as pd

res = pd.read_csv('result.csv')

In [9]:
res.head()

,Unnamed: 0,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,decade
0,0,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en,2000
1,1,Lord You Know,rap,Cam'ron,2004,11882,"{""Cam\\'ron"",""Juelz Santana"",Jaheim}","[Chorus: Jaheim]\nNow Lord you know, just how ...",11,en,en,en,2000
2,2,Its Hot Some Like It Hot,rap,JAY-Z,1999,103549,{},"[Produced by Timbaland]\n\n[Verse 1]\nYo, show...",18,en,en,en,1990
3,3,Time,rap,AZ,2004,7577,"{Nas,Nature}","[Verse 1: Nature]\nAt the ripe age of 24, I do...",8717,en,en,en,2000
4,4,Barry Bonds,rap,Kanye West,2007,280626,"{""Lil Wayne""}",[Verse 1: Kanye West]\nIt's what you all been ...,38,en,en,en,2000
